* transform the data stress from rectangular coordinate system to cylindrical coordinate system

----env 'gmsh_env' is used.

# Read into the stress

import the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.spatial import KDTree
import pandas as pd
import sys

Read into the stress and coordinates in rectangular coordinate system

In [13]:
num_struct_thet = 600
num_struct_zeta = 301

zeta_low_contrl = -0.6 # (mm)
zeta_upp_contrl = +0.6 # (mm)

interva_struct = 0.004 # (mm)
interva_theta = 2*np.pi/num_struct_thet

structured_grid_xls_folder = 'C:\\Abaqus_Works\\roughness_simulation\\cylinder'

rough_or_flat = "rough"


data_csv_file = f"abaqus_cylinder_{rough_or_flat}.csv"
data_xls_file = f"output_stress_cylinder_{rough_or_flat}.xlsx"
node_xls_file = f"nodes_cylinder_{rough_or_flat}.xlsx"

print(f"Current csv file name is {data_csv_file}")
data_csv_path = os.path.join(structured_grid_xls_folder, data_csv_file)

data_all_nodes_string = pd.read_csv(data_csv_path, header=None, skiprows=1)
# print(data_all_nodes_string.shape)
# Column-0 is the node label, Column-1:3 are coordinates , Column-4:9
nodes_struct_data = data_all_nodes_string.to_numpy(dtype=float)
# print(nodes_struct_data.shape)

Current csv file name is abaqus_cylinder_rough.csv


Transform the stress tensor

In [14]:
sum_nodes = nodes_struct_data.shape[0]

# stress_cylind_systm = np.zeros((num_struct_thet, num_struct_zeta, 6))
# nodes_cylind_surf = np.zeros((num_struct_thet, num_struct_zeta),dtype=int)

stress_cylind_systm = np.zeros((num_struct_zeta, num_struct_thet, 6))
nodes_cylind_surf = np.zeros((num_struct_zeta, num_struct_thet),dtype=int)

sigma_rect = np.zeros((3, 3))
Rx_thet = np.zeros((3, 3))

# Z ~ 90deg
Rz_1 = np.array([
    [ 0.0, 1.0, 0.0],
    [-1.0, 0.0, 0.0],
    [ 0.0, 0.0, 1.0]])

# Y ~ -90deg
Ry_2 = np.array([
    [ 0.0, 0.0, 1.0],
    [ 0.0, 1.0, 0.0],
    [-1.0, 0.0, 0.0]])

# X ~ 90deg
Rx_3 = np.array([
    [ 1.0,  0.0, 0.0],
    [ 0.0,  0.0, 1.0],
    [ 0.0, -1.0, 0.0]])


for idx_node in range(sum_nodes):
    node_label = nodes_struct_data[idx_node, 0]
    x = nodes_struct_data[idx_node, 1]
    y = nodes_struct_data[idx_node, 2]
    z = nodes_struct_data[idx_node, 3]

    radius = np.sqrt(x**2 + y**2)
    # if x > + radius:
    #     x = +radius
    # if x < - radius:
    #     x = -radius
    
    if y >= 0.0:
        thet = np.acos(x/radius)
    else:
        thet = np.pi*2 - np.acos(x/radius)
    
    # stress tensor in rectangular coordinate system
    sigma_rect[0, 0] = nodes_struct_data[idx_node, 4] # S11
    sigma_rect[1, 1] = nodes_struct_data[idx_node, 5] # S22
    sigma_rect[2, 2] = nodes_struct_data[idx_node, 6] # S33
    sigma_rect[0, 1] = nodes_struct_data[idx_node, 7] # S12
    sigma_rect[0, 2] = nodes_struct_data[idx_node, 8] # S13
    sigma_rect[1, 2] = nodes_struct_data[idx_node, 9] # S23
    sigma_rect[1, 0] = sigma_rect[0, 1]               # S21
    sigma_rect[2, 0] = sigma_rect[0, 2]               # S31
    sigma_rect[2, 1] = sigma_rect[1, 2]               # S23

    # rotating matrix of theta
    Rx_thet[0, 0] = np.cos(thet)
    Rx_thet[1, 1] = np.cos(thet)
    Rx_thet[2, 2] = 1.0
    Rx_thet[0, 1] = np.sin(thet)
    Rx_thet[0, 2] = 0.0
    Rx_thet[1, 2] = 0.0
    Rx_thet[1, 0] = - Rx_thet[0, 1]
    Rx_thet[2, 0] = - Rx_thet[0, 2]
    Rx_thet[2, 1] = - Rx_thet[1, 2]

    # transform the stress tensor
    R_com = Rx_3 @ Ry_2 @ Rz_1 @ Rx_thet
    sigma_cylin = R_com @ sigma_rect @ R_com.T

    # indexes in the cylindrical surface
    idx_thet = round(thet/interva_theta)
    idx_zeta = round((z - zeta_low_contrl)/interva_struct)

    # nodes_cylind_surf[idx_thet, idx_zeta] = node_label

    # stress_cylind_systm[idx_thet, idx_zeta, 0] = sigma_cylin[0, 0]
    # stress_cylind_systm[idx_thet, idx_zeta, 1] = sigma_cylin[1, 1]
    # stress_cylind_systm[idx_thet, idx_zeta, 2] = sigma_cylin[2, 2]
    # stress_cylind_systm[idx_thet, idx_zeta, 3] = sigma_cylin[0, 1]
    # stress_cylind_systm[idx_thet, idx_zeta, 4] = sigma_cylin[0, 2]
    # stress_cylind_systm[idx_thet, idx_zeta, 5] = sigma_cylin[1, 2]

    nodes_cylind_surf[idx_zeta, idx_thet] = node_label

    stress_cylind_systm[idx_zeta, idx_thet, 0] = sigma_cylin[0, 0]
    stress_cylind_systm[idx_zeta, idx_thet, 1] = sigma_cylin[1, 1]
    stress_cylind_systm[idx_zeta, idx_thet, 2] = sigma_cylin[2, 2]
    stress_cylind_systm[idx_zeta, idx_thet, 3] = sigma_cylin[0, 1]
    stress_cylind_systm[idx_zeta, idx_thet, 4] = sigma_cylin[0, 2]
    stress_cylind_systm[idx_zeta, idx_thet, 5] = sigma_cylin[1, 2]

    if node_label == 90098:
        print(sigma_cylin)
    
steps_name = ['TENSILE_X']

# output the nodes matrix
file_struct_node_path = os.path.join(structured_grid_xls_folder, node_xls_file)
with pd.ExcelWriter(file_struct_node_path, engine='openpyxl') as writer:
    sheet_name = f"node_matrix"
    df = pd.DataFrame(nodes_cylind_surf[:, :])
    df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

# write the strain at the structured grids using numpy
file_struct_data_path = os.path.join(structured_grid_xls_folder, data_xls_file)

# output the stress matrix
ss_comp = ['11', '22', '33', '12', '13', '23']
with pd.ExcelWriter(file_struct_data_path, engine='openpyxl') as writer:
    for i in range(6):  
        sheet_name = f"{steps_name[i // 6]}-S{ss_comp[i % 6]}"
        df = pd.DataFrame(stress_cylind_systm[:, :, i])
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

[[-1.16486562 -7.01317678 -1.3248698 ]
 [-7.01317678  0.12336309 -2.4344593 ]
 [-1.3248698  -2.4344593  -0.63917704]]
